In [11]:
import json
from bs4 import BeautifulSoup

In [12]:
def write_json(bookmarks_data):
  bookmarks_json = json.dumps(bookmarks_data, indent=2)

# Write the JSON data to a file
  with open('data/bookmarks.json', 'w') as file:
      file.write(bookmarks_json)

In [13]:
from urllib.parse import urlparse

def get_favicon_filename(url):
    parsed_url = urlparse(url)
    domain_parts = parsed_url.netloc.split('.')[-2:]
    favicon_filename = '.'.join(domain_parts) + '.favicon.ico'
    return f'/static/assets/icons/{favicon_filename}'

In [14]:
def link(a_tag):
  return {
    'name': a_tag.text,
    'href': a_tag['href'],
    'favicon': get_favicon_filename(a_tag['href']),
    'add_date': a_tag['add_date']
  }

In [15]:
def parse_bookmarks(parent):
  children = []
  
  for child in parent.children:
    if child.name == 'a':
      children.append(link(child))
    if child.name == 'h3':
      dl = child.find_next_sibling('dl')
      children.append({
        'name': child.text,
        'contents': parse_bookmarks(dl)
      })

  return children

with open('data/just_bookmarks_5_6_24_no_icons.html', 'r', encoding='utf-8') as file:
    html_content = file.read()
soup = BeautifulSoup(html_content, 'html.parser')
bookmarks = parse_bookmarks(soup.find('dl'))
with open('./data/bookmarks.json', 'w', encoding='utf-8') as f:
  json.dump(bookmarks[0]['contents'], f, ensure_ascii=False, indent=2)